# **PROBLEMA DE NEGOCIO**


---



La urgencia por detectar fraudes en transacciones móviles de dinero ha llevado a una empresa del segmento Fintech a buscar soluciones innovadoras. Como científicos de datos hemos sido convocados para desarrollar un modelo de machine learning que pueda distinguir de manera precisa entre transacciones legítimas y fraudulentas, estableciendo así un estándar de seguridad en el sector financiero móvil global.

### **Tus tareas principales serán:**

**1. Preprocesamiento de Datos:** Realizar limpieza de datos, manejar valores faltantes, codificación de variables categóricas y normalización/escalado de datos.

**2. Exploración de Datos:** Analizar y comprender el conjunto de datos proporcionado, identificar variables llaves y realizar visualizaciones para entender las relaciones entre las variables y seleccionar las características relevantes.

**3. Construcción de Modelos:** Experimentar con algunos algoritmos de machine learning como Regresión Logística, Árboles de Decisión, Random Forest, Naive Bayes, entre otros.

**4. Evaluación y Selección del Modelo:** Evaluar los modelos utilizando métricas como precisión, recall, área bajo la curva ROC, y F1-score. Seleccionar el modelo con el mejor rendimiento para la detección de transacciones bancarias fraudulentas.

# **1. Configuración del Ambiente**


---


In [1]:
import gdown
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
#!pip install -U imbalanced-learn
#!pip install ydata-profiling
from ydata_profiling import ProfileReport
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve
global df_banco, resultados

# **2. Preprocesamiento de Datos**


---


In [2]:
df_banco =pd.read_csv('data/6MData.csv', sep = ',')

In [3]:
df_banco

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


##### **Nota:Transactions which are detected as fraud are cancelled, so for fraud detection these columns (oldbalanceOrg, newbalanceOrig, oldbalanceDest, newbalanceDest ) must not be used.**

In [4]:
df_banco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [5]:
# Eliminar las columnas especificadas
columnas_a_eliminar = ['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
df_banco = df_banco.drop(columnas_a_eliminar, axis=1)


In [6]:
df_banco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   nameDest        object 
 5   isFraud         int64  
 6   isFlaggedFraud  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 339.8+ MB


In [7]:
# Contar los valores no nulos en cada columna
num_datos_por_columna = df_banco.count()

# Imprimir el resultado
print(num_datos_por_columna)

step              6362620
type              6362620
amount            6362620
nameOrig          6362620
nameDest          6362620
isFraud           6362620
isFlaggedFraud    6362620
dtype: int64


In [8]:
df_banco.describe()

,step,amount,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,1.000000e+00,1.000000e+00


In [9]:
# Tomar los primeros 100,000 registros del DataFrame
df_banco_muestra = None


In [11]:
def procesar_datos(df):
    # Definir variables globales
    global df_banco_muestra
    
    # Tomar una muestra de 100,000 registros del DataFrame original
    df_banco_muestra = df.head(2000000)
    
    # Crear la variable type2
    df_banco_muestra['type2'] = df_banco_muestra['nameOrig'].str[0] + df_banco_muestra['nameDest'].str[0]
    
    # Realizar One Hot Encoding para las columnas type y type2
    df_banco_muestra = pd.get_dummies(df_banco_muestra, columns=['type', 'type2'], dtype=int)
    
    # Eliminar columnas prescindibles
    columnas_a_eliminar = ['nameOrig', 'nameDest', 'isFlaggedFraud']
    df_banco_muestra = df_banco_muestra.drop(columns=columnas_a_eliminar)
    
    # Eliminar registros duplicados y guardar el resultado en df_banco_muestra
    df_banco_muestra.drop_duplicates(inplace=True)
    
    # Tratar los valores nulos
    df_banco_muestra.dropna(inplace=True)  # Eliminar registros con valores nulos
    


In [12]:
# Determinar el tamaño de cada parte
tamano_parte = 1000000  # Procesar de 1 millón en 1 millón

# Definir una lista para almacenar los DataFrames procesados
dataframes_procesados = []

# Iterar sobre el DataFrame original en partes
for i in range(0, len(df_banco), tamano_parte):
    # Obtener la parte actual del DataFrame
    parte_actual = df_banco.iloc[i:i+tamano_parte]
    
    # Procesar la parte actual usando la función procesar_datos()
    procesar_datos(parte_actual)
    
    # Agregar la parte procesada a la lista
    dataframes_procesados.append(df_banco_muestra)
    
# Concatenar todos los DataFrames procesados en uno solo
df_banco_final = pd.concat(dataframes_procesados, ignore_index=True)


In [13]:
df_banco_final

,step,amount,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,type2_CC,type2_CM
0,1,9839.64,0,0,0,0,1,0,0,1
1,1,1864.28,0,0,0,0,1,0,0,1
2,1,181.00,1,0,0,0,0,1,1,0
3,1,181.00,1,0,1,0,0,0,1,0
4,1,11668.14,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
6355046,743,339682.13,1,0,1,0,0,0,1,0
6355047,743,6311409.28,1,0,0,0,0,1,1,0
6355048,743,6311409.28,1,0,1,0,0,0,1,0
6355049,743,850002.52,1,0,0,0,0,1,1,0


In [14]:
df_banco_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6355051 entries, 0 to 6355050
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   step           int64  
 1   amount         float64
 2   isFraud        int64  
 3   type_CASH_IN   int32  
 4   type_CASH_OUT  int32  
 5   type_DEBIT     int32  
 6   type_PAYMENT   int32  
 7   type_TRANSFER  int32  
 8   type2_CC       int32  
 9   type2_CM       int32  
dtypes: float64(1), int32(7), int64(2)
memory usage: 315.2 MB


In [15]:
# Contar los valores no nulos en cada columna
num_datos_por_columna = df_banco_final.count()

# Imprimir el resultado
print(num_datos_por_columna)

step             6355051
amount           6355051
isFraud          6355051
type_CASH_IN     6355051
type_CASH_OUT    6355051
type_DEBIT       6355051
type_PAYMENT     6355051
type_TRANSFER    6355051
type2_CC         6355051
type2_CM         6355051
dtype: int64


In [16]:
df_banco_final.isnull().sum()

step             0
amount           0
isFraud          0
type_CASH_IN     0
type_CASH_OUT    0
type_DEBIT       0
type_PAYMENT     0
type_TRANSFER    0
type2_CC         0
type2_CM         0
dtype: int64

In [17]:
df_banco_final.to_csv('data/df_banco_final.csv.gz', sep=';', index=False, compression='gzip')